In [1]:
# coding = utf-8

import torch
from torch import nn
import os
import data_load_for_Transformer as dlft
import BERT_pretraining_single
import BERTModel
import Utility
import imp
imp.reload(BERTModel)

<module 'BERTModel' from 'e:\\ML\\chatbot\\BERTChatbot\\BERTModel.py'>

In [2]:
print("Initiating the hyperparameters...")
num_hiddens, num_layers, dropout, batch_size, num_steps = 32, 2, 0.1, 128, 16
lr, num_epochs, device = 0.001, 2000, Utility.try_gpu()
ffn_num_input, ffn_num_hiddens, num_heads = 32, 64, 4
key_size, query_size, value_size = 32, 32, 32
norm_shape = [32]

Initiating the hyperparameters...


In [3]:
print("Building the vocabulary...")
train_iter, vocab = dlft.load_data_xhj_for_Transformer(batch_size, num_steps)

Building the vocabulary...
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\RuiLi\AppData\Local\Temp\jieba.cache
Using 'word' as tokens
Loading model cost 0.560 seconds.
Prefix dict has been built succesfully.
total training size= 3102


In [4]:
print(vocab.to_tokens([0,1,2,3,4,5,6,7,8,9]))

['<unk>', '<pad>', 'mask2', '<bos>', '<eos>', '你', '我', '的', '了', '是']


In [5]:
print("Rebuilding the Model...")
bert = BERTModel.BERTModel(len(vocab), num_hiddens=num_hiddens, norm_shape=norm_shape, 
    ffn_num_input=ffn_num_input, ffn_num_hiddens=ffn_num_hiddens, num_heads=num_heads, 
    num_layers=num_layers, dropout=dropout, key_size=key_size, query_size=query_size, 
    value_size=value_size, hid_in_features=value_size, mlm_in_features=value_size, 
    nsp_in_features=value_size)
bert = BERT_pretraining_single.load_bert(bert, device )
# for v in bert.parameters():
#     v.requires_grad = False
encoder = BERTModel.BERTEncoder(bert, hid_in_features=value_size, num_outputs=value_size)
decoder = BERTModel.TransformerDecoder(
    len(vocab), key_size, query_size, value_size, num_hiddens,
    norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
    num_layers, dropout)
net = BERTModel.EncoderDecoder(encoder, decoder)

try:
    checkpoint_prefix = os.path.join("model_data/model_bert.pt")
    checkpoint = torch.load(checkpoint_prefix)
    net.load_state_dict(checkpoint['model_state_dict'])
except Exception as e:
    print("Can not load the model with error:", e)

print("Ready to working...")
def predict(src_sentence):
    return BERTModel.predict_seq2seq(net, src_sentence, vocab, vocab, num_steps,
                    device, save_attention_weights=False)


Rebuilding the Model...
Ready to working...


In [6]:
BERTModel.train_bert(net, train_iter, lr, num_epochs, batch_size, vocab, device)

KeyboardInterrupt: 

In [19]:
predict("你好")

src ['你', '好']
src_tokens= [4, 13, 3]
tp_src_tokens [4, 13, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
enc_outputs tensor([[[ 9.7231e-01,  9.4238e-01,  8.5993e-01, -8.1902e-01,  9.6128e-01,
          -9.3587e-01, -8.0915e-01, -9.1387e-01, -9.5669e-01,  8.9776e-01,
           8.0449e-01, -9.6980e-01,  9.3785e-01, -8.9145e-01,  9.6813e-01,
           9.4570e-01, -8.0404e-01,  9.4238e-01, -9.2957e-01,  9.1584e-01,
          -7.4726e-01,  6.4430e-01,  7.5854e-02,  8.8592e-01, -9.5980e-01,
           8.6292e-01, -8.3160e-01,  9.2398e-01, -4.6249e-01, -9.4882e-01,
           8.0638e-01,  9.1028e-01],
         [ 9.6996e-01,  9.4125e-01,  8.6895e-01, -8.1404e-01,  9.7040e-01,
          -9.3767e-01, -8.0871e-01, -9.3105e-01, -9.3480e-01,  8.7052e-01,
           7.6929e-01, -9.6400e-01,  9.4805e-01, -8.9169e-01,  9.6769e-01,
           9.4685e-01, -8.3243e-01,  9.4473e-01, -9.4405e-01,  9.0633e-01,
          -7.5461e-01,  7.1681e-01,  5.0132e-02,  8.8928e-01, -9.5430e-01,
           8.3614e-01, -

''